In [1]:
from langchain_community.document_loaders import CSVLoader

In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path=r"C:\Users\praneeth\Downloads\healtcare.csv",metadata_columns=['TestCaseID', 'RequirementID','SourceFormat', 'TestType', 'ComplianceStandard', 'IntegratedTool', 'Status'])
data = loader.load()
print(data)
                          

[Document(metadata={'source': 'C:\\Users\\praneeth\\Downloads\\healtcare.csv', 'row': 0, 'TestCaseID': 'TC001', 'RequirementID': 'REQ-HS-01', 'SourceFormat': 'PDF', 'TestType': 'Security', 'ComplianceStandard': 'ISO 27001', 'IntegratedTool': 'Jira', 'Status': 'Approved'}, page_content='RequirementText: The system must allow physicians to securely log in using two-factor authentication (2FA).\nGeneratedTestCase: Verify that a user with valid credentials is prompted for a 2FA code from their authenticator app upon login.\nExpectedResult: User successfully logs in after providing the correct 2FA code.'), Document(metadata={'source': 'C:\\Users\\praneeth\\Downloads\\healtcare.csv', 'row': 1, 'TestCaseID': 'TC002', 'RequirementID': 'REQ-HS-01', 'SourceFormat': 'PDF', 'TestType': 'Security', 'ComplianceStandard': 'ISO 27001', 'IntegratedTool': 'Jira', 'Status': 'Approved'}, page_content="RequirementText: The system must allow physicians to securely log in using two-factor authentication (2FA

In [3]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model="text-embedding-004")

c:\Users\praneeth\praneethproject_AI\hackaton_Project\.venv\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [4]:
print(data)
list_data=[]
for i in data:
    list_data.append([i.metadata['source'],i.page_content])
print(list_data)  # this is for refercence only this block no need for this operation

[Document(metadata={'source': 'C:\\Users\\praneeth\\Downloads\\healtcare.csv', 'row': 0, 'TestCaseID': 'TC001', 'RequirementID': 'REQ-HS-01', 'SourceFormat': 'PDF', 'TestType': 'Security', 'ComplianceStandard': 'ISO 27001', 'IntegratedTool': 'Jira', 'Status': 'Approved'}, page_content='RequirementText: The system must allow physicians to securely log in using two-factor authentication (2FA).\nGeneratedTestCase: Verify that a user with valid credentials is prompted for a 2FA code from their authenticator app upon login.\nExpectedResult: User successfully logs in after providing the correct 2FA code.'), Document(metadata={'source': 'C:\\Users\\praneeth\\Downloads\\healtcare.csv', 'row': 1, 'TestCaseID': 'TC002', 'RequirementID': 'REQ-HS-01', 'SourceFormat': 'PDF', 'TestType': 'Security', 'ComplianceStandard': 'ISO 27001', 'IntegratedTool': 'Jira', 'Status': 'Approved'}, page_content="RequirementText: The system must allow physicians to securely log in using two-factor authentication (2FA

In [5]:
from langchain_community.vectorstores import Chroma

In [6]:
vector_store = Chroma.from_documents(
    documents=data, # list conatins objects 
    embedding=embeddings,
    persist_directory="./chroma_db_testcases" # Optional: to save the database
)

In [7]:
retriver = vector_store.as_retriever(search_type="similarity",search_kwargs={"k": 3}) #search_kwargs={"k": 3}
#this is noramal retriver

In [8]:
retriver_results = retriver.invoke("test case",
    filter={"TestCaseID": "TC007"}) # if you want individual you can can do it
# pass the message if you want to test it
print(retriver_results) # just for test case 

[Document(metadata={'source': 'C:\\Users\\praneeth\\Downloads\\healtcare.csv', 'IntegratedTool': 'Azure DevOps', 'SourceFormat': 'XML', 'row': 6, 'RequirementID': 'REQ-DATA-08', 'TestCaseID': 'TC007', 'TestType': 'Functional', 'ComplianceStandard': 'N/A', 'Status': 'Approved'}, page_content="RequirementText: The system must support importing patient data using the HL7 V2 standard.\nGeneratedTestCase: Prepare a valid HL7 V2 message with new patient data. Use the import tool to process the message.\nExpectedResult: The new patient's record is created in the system with all data from the HL7 message correctly populated."), Document(metadata={'Status': 'Approved', 'row': 6, 'TestType': 'Functional', 'ComplianceStandard': 'N/A', 'TestCaseID': 'TC007', 'SourceFormat': 'XML', 'source': 'C:\\Users\\praneeth\\Downloads\\healtcare.csv', 'IntegratedTool': 'Azure DevOps', 'RequirementID': 'REQ-DATA-08'}, page_content="RequirementText: The system must support importing patient data using the HL7 V2

In [9]:
%pip install lark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from langchain.chat_models import init_chat_model
llm=init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai", temperature=0) 


In [11]:

from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# 1. Define metadata information for all columns in your CSV.
metadata_field_info = [
    AttributeInfo(
        name="TestCaseID",
        description="A unique identifier for a specific test case, such as 'TC001' or 'TC105'",
        type="string",
    ),
    AttributeInfo(
        name="RequirementID",
        description="The identifier for the requirement that the test case is based on, such as 'REQ-SEC-01'",
        type="string",
    ),
    AttributeInfo(
        name="SourceFormat",
        description="The format of the source document for the requirement, like 'PDF', 'Word', or 'XML'",
        type="string",
    ),
    AttributeInfo(
        name="TestType",
        description="The category or type of the test, such as 'Security', 'Functional', 'Performance', or 'Compliance'",
        type="string",
    ),
    AttributeInfo(
        name="ComplianceStandard",
        description="The specific compliance standard the test case is designed to meet, for example 'GDPR', 'ISO 27001', or 'FDA21CFRPart11'",
        type="string",
    ),
    AttributeInfo(
        name="IntegratedTool",
        description="The project management or testing tool integrated with the test case, such as 'Jira', 'Azure DevOps', or 'Polarion'",
        type="string",
    ),
    AttributeInfo(
        name="Status",
        description="The current status of the test case, for example 'Approved', 'Needs Review', or 'Auto-Generated'",
        type="string",
    ),
]
document_content_description = "A test case for a healthcare software system"
# Note: RequirementText, GeneratedTestCase, and ExpectedResult are typically not included
# here because they contain long, free-form text that is not ideal for exact filtering.
# The retriever will use semantic search on these fields automatically.





In [12]:
# 2. Set up the Self-Querying Retriever
def retrival_function(n):
    self_query_retriever = SelfQueryRetriever.from_llm(
    llm,
    vector_store,
    document_content_description,
    metadata_field_info,
    verbose=True, # Set to True to see the generated query
     search_kwargs={"k": n},  # <-- Add this line to get up to 20 results
)
    return self_query_retriever
    

In [14]:
self=retrival_function(4)
self.get_relevant_documents("Tell me about TC001")


C:\Users\praneeth\AppData\Local\Temp\ipykernel_15016\3209673161.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  self.get_relevant_documents("Tell me about TC001")


[Document(metadata={'row': 29, 'source': 'C:\\Users\\praneeth\\Downloads\\healtcare.csv'}, page_content="TestCaseID: TC030\nRequirementID: REQ-ISO-02\nRequirementText: There must be a documented procedure for handling customer complaints.\nSourceFormat: PDF\nGeneratedTestCase: Search the quality management system documentation for a 'Complaint Handling Procedure'.\nExpectedResult: A formal document outlining the process for logging, investigating, and resolving customer complaints is found.\nTestType: Compliance\nComplianceStandard: ISO 13485\nIntegratedTool: Azure DevOps\nStatus: Auto-Generated"),
 Document(metadata={'source': 'C:\\Users\\praneeth\\Downloads\\healtcare.csv', 'row': 29}, page_content="TestCaseID: TC030\nRequirementID: REQ-ISO-02\nRequirementText: There must be a documented procedure for handling customer complaints.\nSourceFormat: PDF\nGeneratedTestCase: Search the quality management system documentation for a 'Complaint Handling Procedure'.\nExpectedResult: A formal d

In [15]:
from langchain_core.prompts import ChatPromptTemplate

In [17]:
system_message = """You are an expert Q&A assistant for a software quality assurance team. Your purpose is to provide precise answers.

- First, check the provided Context. If the Context is not empty, answer the user's question based ONLY on that information.
- Do not make up information or answer questions outside of the given Context.
- If user ask you to summarize the information then summarie
- if the information is not there print that "I cannot find that information in the provided data."

"""

In [18]:

human_message = """
Context:
{context}

---

Question:
{question}
"""

In [19]:
Prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", human_message)
]) #Used to Create the Prompt Template
type(Prompt_template)

langchain_core.prompts.chat.ChatPromptTemplate

In [20]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [70]:
messsage_prompt="summarise the  data"

In [71]:
K_value=llm.invoke(f'''You are an expert at classifying user queries. Please classify the following query as either 'lookup' or 'aggregation'.

A 'lookup' query asks for a specific item, like "TC046" or "REQ-HS-01".
An 'aggregation' query asks for a summary, count, or a complete list of items.

**IMPORTANT RULE: If the query contains a specific ID (like TC followed by numbers), it is ALWAYS a 'lookup', even if it also contains words like 'summary'.**

-- Examples --
Query: "show the information for TC046"
Classification: lookup so in content return 4

Query: "how many test cases are approved?"
Classification: aggregation so in content return 250

Query: "give me a summary of all the GDPR compliance standards"
Classification: aggregation so in content return 250

Query: "summary of TC005"
Classification: lookup so in content return 4

if it is a lookup return only number '4'
if it is a aggregation return only number '250'
else default number 5
In result produce only the number value

-- User Query --
Query: "{messsage_prompt}"
Classification:'''
)
print(K_value.content)

250



In [72]:
rag_chain = (

{"context":retrival_function(int(K_value.content)), "question": RunnablePassthrough()}
| Prompt_template
| llm
| StrOutputParser()

) 

In [73]:
respose=rag_chain.invoke(messsage_prompt)

In [74]:
print(respose)

Here is a summary of the provided data:

The data consists of test cases for a healthcare system, covering various aspects such as:

*   **Functional Testing:** Generating reports (annual summaries, medication administration records, overdue patients), parsing patient allergy information from CCDA files, and recording/displaying patient vital signs.
*   **Security Testing:** Ensuring data encryption at rest, role-based data access, and secure data export.
*   **Performance Testing:** Evaluating the speed of patient search functionality and the scalability of the system under concurrent users.
*   **Compliance Testing:** GDPR compliance for data deletion, data storage locations, and general readiness, as well as ISO 9001 and ISO 13485 compliance for design review processes.

The test cases are linked to specific requirements and utilize various integrated tools such as Polarion, Azure DevOps, and Jira. The source format for requirements varies (Word, XML, Markup, PDF), and the test case